In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import kagglehub
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

from trainer import Trainer

#import Dataloader
from dataloader import ImageColorizerDataset

#import architectures
from neural_nets import ImageColorizer, ImageSuperRes, UNet, UNetRes, ImageColorizerLAB, ColorizeNet, count_params
from grid_search import run_grid_search, plot_results

/Users/gabrielmindlberger/Library/Mobile Documents/com~apple~CloudDocs/FH-AI/Sem2/ML/PRO/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/gabrielmindlberger/Library/Mobile Documents/com~apple~CloudDocs/FH-AI/Sem2/ML/PRO/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Download latest version
IMAGE_PATH = kagglehub.dataset_download("arnaud58/landscape-pictures")
print("Path to dataset files:", IMAGE_PATH)

Path to dataset files: /Users/gabrielmindlberger/.cache/kagglehub/datasets/arnaud58/landscape-pictures/versions/2


In [4]:
# 2. Device config
device = torch.device(
    'mps' if torch.backends.mps.is_available()
    else 'cuda' if torch.cuda.is_available()
    else 'cpu'
)

print(f'Using device: {device}')

Using device: mps


In [5]:
# 3. Hyperparameters
img_height, img_width = 128, 128
batch_size            = 32
num_epochs            = 10
learning_rate         = 1e-3

# 3.1 Hyperparameters with Grid-Search
grid = {
    'img_size': [(96,96), (128,128), (180,180)],
    'lr':       [1e-2, 1e-3, 1e-4],
    'bs':       [16, 32],
    'epochs':   [1, 2],
}
df_results = run_grid_search(IMAGE_PATH, grid)
plot_results(df_results)

In [6]:
# 1. Dataset laden
dataset   = ImageColorizerDataset(IMAGE_PATH, img_height, img_width)
n_total   = len(dataset)

# 2. Fraktionen definieren
val_frac  = 0.1   # 20 % für Validation
test_frac = 0.01   # 10 % für Test
train_frac = 1.0 - val_frac - test_frac

# 3. Größen berechnen
n_val   = int(n_total * val_frac)
n_test  = int(n_total * test_frac)
n_train = n_total - n_val - n_test  # Rest für Training

# 4. Splitten
train_ds, val_ds, test_ds = random_split(
    dataset,
    [n_train, n_val, n_test],
    generator=torch.Generator().manual_seed(42)  # für Reproduzierbarkeit
)

# 5. DataLoader anlegen
train_loader = DataLoader(train_ds,   batch_size=batch_size, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_ds,     batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_ds,    batch_size=batch_size, shuffle=False, num_workers=4)

# 6. Größen-Check
print(f'Total images:  {n_total}')
print(f'Train images:  {len(train_ds)}')
print(f'Val images:    {len(val_ds)}')
print(f'Test images:   {len(test_ds)}')


Total images:  4319
Train images:  3845
Val images:    431
Test images:   43


In [7]:
model = ImageColorizer(1, 2)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

count_params(model)

ImageColorizer(
  (enc1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (up1): Upsample(scale_factor=2.0, mode='nearest')
  (dec1): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (up2): Upsample(scale_factor=2.0, mode='nearest')
  (dec2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (final): Conv2d(64, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (sigmoid): Sigmoid()
)


739650

In [ ]:
trainer = Trainer(model = model, train_loader= train_loader, val_loader=val_loader, criterion=criterion, optimizer=optimizer, device=device, validate=True)
trainer.fit(num_epochs)

# 6. Lernkurve anzeigen
trainer.plot_history()

  Train:   0%|          | 0/121 [00:00<?, ?it/s]

In [ ]:
trainer.show_examples(test_loader)